# Entrega Final 

## Simulación de Eventos Discretos

#### Departamento de Computación - Facultad de Ciencias Exactas y Naturales - Universidad de Buenos Aires

Cursada 2021

_Confalonieri, Gisela - Lucero, Emiliano_

----

# Resumen
En este notebook presentamos un conjunto de herramientas desarrolladas con el fin de facilitar la utilización del simulador CD++ y el análisis de los resultados dentro del entorno de JupyterLab. El enfoque está puesto especialmente en el dictado del curso de Simulación de Eventos Discretos, buscando minimizar la cantidad de contenido no específico de DEVS y CellDEVS, de manera que quienes tomen el curso puedan enfocarse en la forma de modelado más que en el código que permite correr las simulaciones desde un Jupyter Notebook y observar los resultados obtenidos.

Para el manejo del simulador CD++ hemos desarrollado un conjunto de comandos tipo magic[1] que pueden ser invocados desde una linea de una celda de código de un Jupyter Notebook. También presentamos comandos magic a la par de un conjunto de métodos específicos para el procesamiento de los resultados de las simulaciones ejecutadas.

# Herramientas desarrolladas

El conjunto de comandos magics y métodos que hemos desarrollado se encuentran en los archivos `00-cdpp_parsing_plotting.py` y `01-cdpp_magics.py` que acompañan esta entrega. Los mismos deben ubicarse en la carpeta `~/.ipython/profile_{NAME}/startup/` donde `{NAME}` es el nombre del perfil de IPython sobre el cual se va a trabajar. Por defecto es _default_(`profile_default`). De esta manera, cada vez que se inicie Jupyter Lab se podrán utilizar los comandos y métodos definidos. 

Proveemos de un script `install.sh` que se ocupa de copiar los archivos en el directorio mencionado, basta con correr `./install.sh` desde una terminal. Es posible que se requiera asignar permisos de ejecución al script, lo cual se logra corriendo `chmod +x install.sh` desde la terminal. 

En caso de haber iniciado JupyterLab antes de hacer esta copia, deberá reiniciarse.

## Comandos magic

Hemos desarrollado una colección de funciones magics para facilitar el uso del simulador CD++ en JupyterLab. 

Todos los magics definidos son magics de línea. Esto quiere decir que considera sólo una línea de una celda de código, permitiendo continuar con código Python (u otros magics de línea) en el resto de la celda.

Para consultar los comandos disponibles, ofrecemos el magic `%lscdpp`:

In [ ]:
%lscdpp

A su vez, se puede consultar la definición de cada comando utilizando el símbolo `?` luego del comando en cuestión. Por ejemplo, veamos la definición del comando `%cdpp_run`:

In [ ]:
%cdpp_run?

## Magics y métodos para el procesamiento de resultados

Para procsear los archivos resultantes de las simulaciones se proveen algunos comandos magics y otros métodos. Los comandos magics desarrollados a estos efectos reciben el path a los archivos de salida o de log que se producen al ejecutar una simulación y parsean su contenido en un dataframe de pandas. Estos dataframes quedan disponibles para ser consultados y utilizados en el resto del notebook. También desarrollamos algunos métodos para graficar estos dataframes. 

# Ejemplos de uso

A continuación mostraremos la utilización de las nuevas herramientas, tomando como base algunos ejemplos dados en clase.

## Inicialización

Comenzamos inicializando algunas variables de entorno que aprovecha CD++ (esto puede tardar unos segundos).

In [ ]:
# inicializamos variables
%cdpp_init /home/gisela

Podemos notar que el magic `%cdpp_init` recibe como parámetro el path a la carpeta que contiene la carpeta `SED`, la cual contiene el simulador CD++. Como salida de este comando, se observa un diccionario de variables de entorno que pueden ser utilizadas en cualquier celda de código del notebook actual. Para acceder a estas variables, basta con escribir `CDPP_PATHS[VAR]` siendo `VAR` la variable de entorno a la que se quiere acceder.

## Ejemplo 1: Generador de pulsos

Tomaremos el ejemplo **Pulse** en `examples/pulse/` dado en clase. 

Primero establecemos el path al proyecto sobre el cual trabajar, de la siguiente manera:

In [ ]:
%cdpp_set_project -c examples/pulse

Una vez hecho esto, podemos, por ejemplo, revisar el archivo `pulse.ma` en `examples/pulse/model/` que incluye el atómico `pulse`:

In [ ]:
%cdpp_show model/pulse.ma

Podemos ver también el contenido del archivo `reg.cpp`, donde se ve cómo se registra el modelo en el kernel del simulador:

In [ ]:
%cdpp_show src/reg.cpp

Así, con el magic `%cdpp_show` podemos mostrar en el notebook cualquier archivo de texto.

### Simulación con CD++

Teniendo listo nuestro modelo, vamos a compilar el simulador (esto puede tardar unos segundos):

In [ ]:
%cdpp_compile

Si nos interesa, podemos observar el archivo de eventos de entrada `pulse.ev`:

In [ ]:
%cdpp_show model/pulse.ev

Ahora ejecutemos el ejemplo y veamos los resultados. Para ejecutar la simulación utilizamos el comando `%cdpp_run` seguido de los mismos parámetros que se proveen a cd++ en un entorno bash:

In [ ]:
%cdpp_run -m model/pulse.ma \
          -e model/pulse.ev \
          -l pulse.log \
          -o pulse.out

### Análisis de los resultados

Graficamos los resultados que obtuvimos en el archivo `pulse.out`. Para esto utilizamos el magic `%parse_out_ev` para generar el dataframe, y el método `do_chart()` para generar un gráfico:

In [ ]:
# generamos el dataframe
salida_pulse = %parse_out_ev pulse.out

# generamos un gráfico
do_chart(salida_pulse)

Como se puede ver, el magic `%parse_out_ev` recibe el nombre del archivo a parsear, y su salida puede asignarse a una variable para poder trabajarlo luego. Veamos cómo luce el dataframe:

In [ ]:
salida_pulse

El método `do_chart()` por defecto realiza un gráfico de línea del campo valor en función del campo tiempo. También se pueden realizar otros tipos de gráficos con `do_chart()`, los veremos en los siguientes ejemplos, pero si se necesitase hacer otro gráfico u otros tratamientos sobre el dataframe obtenido de la salida de la simulación, se puede utilizar el resultado de `%parse_out_ev`. También puede extenderse `do_chart()` para que soporte otros tipos de gráfico.

## Ejemplo 2: Semáforo (autómata temporizado)

Este ejemplo también fue dado durante las clases. El modelo fue descargado y ubicado en la carpeta `examples`.

Comenzamos estableciendo el directorio:

In [ ]:
%cdpp_set_project -c examples/semaforo

Y de la misma manera que en el ejemplo anterior, podemos ver el contenido de los archivos de ese directorio. Por ejemplo, veamos los archivos `trafficlights.ma`, `trafficlights.cpp` y `trafficlights.h`:

### Simulación con CD++

Vamos a compilar el simulador:

In [ ]:
%cdpp_compile

Y ya podemos ejecutar la simulación:

In [ ]:
%cdpp_run -m model/trafficlights.ma \
          -l trafficlights.log \
          -o trafficlights.out \
          -t 00:03:00:00

### Análisis de los resultados:

Vamos a parsear el contenido del archivo `trafficlights.out` y a graficar los eventos. En este caso, especificaremos que queremos obtener un gráfico de tipo _step_:

In [ ]:
salida_sem = %parse_out_ev trafficlights.out

do_chart(salida_sem,"step")

Nuevamente, el dataframe queda disponible para ser consultado y utilizado en el notebook:

In [ ]:
salida_sem

## Ejemplo 3: Modelo _Simple Network_

Comenzamos estableciendo el directorio:

In [ ]:
%cdpp_set_project -c examples/SimpleNetwork

A partir de este momento pueden consultarse los archivos del modelo que se precisen utilizando el magic `%cdpp_show`, tal como lo hicimos anteriormente.

### Simulador CD++

Compilemos el simulador para este modelo:

In [ ]:
%cdpp_compile

Ahora nos paramos en donde se encuentra el ejecutable del simulador y corremos $20~\text{s}$ de la simulación:

In [ ]:
%cdpp_run -m model/simpleNetwork.ma \
         -l simpleNetwork.log \
         -o simpleNetwork.out \
         -t 00:02:00:00

## Análisis de los resultados

Ahora veamos los resultados:

In [ ]:
salida_sn = %parse_out_ev simpleNetwork.out
do_chart(salida_sn,"stem",['out','generator_out'])

Como podemos observar, graficamos los eventos de salida del generador y los eventos de salida de la red, diferenciándolos por color. Esto fue posible indicando el tipo de gráfico _stem_ junto con una lista de los puertos presentes en el archivo `simpleNetwork.out` que nos interesaba distinguir. Veamos cómo luce el dataframe:

In [ ]:
salida_sn

# Cell-DEVS

Veamos ahora algunos ejemplos con Cell-DEVS. Ya que se utiliza el mismo simulador CD++, la inicialización de variables que se hace con `%cdpp_init` también contempla este caso. El resto de magics que estuvimos utilizando también sirven para Cell-DEVS, permitiendo utilizar los parámetros específicos del simulador.

Como ya inicializamos las variables necesarias, podemos consultar la ayuda de CD++ de esta manera:

In [ ]:
%cdpp_help

Para compilar las herramientas auxiliares de CD++, como _Drawlog_, utilizamos el comando `%cdpp_compile_tools` (esto puede tardar unos segundos). 

In [ ]:
%cdpp_compile_tools

También podemos acceder a la ayuda de _Drawlog_ con `%drawlog_help`:

In [ ]:
%drawlog_help

## Ejemplo 4: Fire

Este es un ejemplo de propagación de fuego, visto en clase.

Primero establecemos el path al proyecto sobre el cual trabajar y compilamos el simulador:

In [ ]:
%cdpp_set_project examples/cell-devs/fire/
%cdpp_compile

Ahora veamos cómo descargar el modelo del repositorio de modelos y guardarlo con en el path relativo a la carpeta del proyecto indicado:

In [ ]:
%cdpp_download_carleton fire.zip

Y ahora vamos a descomprimir el .zip que contiene el modelo:

In [ ]:
%cdpp_unzip fire.zip

Tal como mostramos más arriba, podemos ver el contenido de los archivos del modelo y los valores iniciales de las celdas:

In [ ]:
%cdpp_show FireMA.ma

In [ ]:
%cdpp_show fire.val

### Simulación con CD++

Vamos a simular 16 min de la propagación del fuego:

In [ ]:
%cdpp_run -m FireMA.ma   \
        -l Fire.log \
        -t 00:16:00:000

### Análisis de los resultados

Veamos el contenido de los archivos `.log`. Esto puede tardar algunos segundos en caso de tener un modelo muy complejo con muchos archivos de log.

In [ ]:
log_fire = %parse_log Fire.log

Veamos cómo lucen los datos parseados por el magic `%parse_log`:

In [ ]:
log_fire

Como se puede observar, se trata de un diccionario cuyas claves son los componentes y celdas del modelo, y donde el valor de una clave es el parseo en un dataframe de su respectivo archivo de log:

In [ ]:
# veo las claves del diccionario
log_fire.keys()

In [ ]:
# veo el dataframe de una de las entradas del diccionario
log_fire['ParallelRoot']

In [ ]:
# veo el dataframe de una de las entradas del diccionario
log_fire['forestfire(0,0)']

Podemos observar que las columnas de los dataframes no están nombradas, y que las filas contienen distinta cantidad de valores dependiendo del valor en la columna 3 (el tipo de mensaje). Para refinar el análisis de estos dataframes, proveemos el método `filter_and_name()` que dados un dataframe y un tipo de mensaje, filtra las filas correspondientes a dicho tipo y acota y renombra las columnas consecuentemente:

In [ ]:
forestfire_tipo_Y = filter_and_name(log_fire["forestfire"],'Y')
forestfire_tipo_Y

### Visualización usando drawlog

Utilicemos la herramienta `drawlog` para visualizar los resultados. Para esto proveemos el magic `%drawlog_run`, al que le debemos pasar los mismos parámetros que al ejecutar `drawlog` desde consola:

In [ ]:
%drawlog_run mFireMA.ma   \
             -lFire.log \
             -cforestfire \
             -i00:00:30:000 \
             -0 \
             -w3\
             -p0

## Ejemplo 5: Difusión del calor 2D

Este ejemplo visto en clase modela la difusión de calor en una barra de material en 2D usando un autómata celular.

Primero establecemos el path al proyecto sobre el cual trabajar y compilamos el simulador:

In [ ]:
%cdpp_set_project examples/cell-devs/2dheat_diffusion/
%cdpp_compile

Ahora descarguemos el modelo a la carpeta indicada y descomprimamos el .zip:

In [ ]:
%cdpp_download_carleton 2dheat_diffusion.zip
%cdpp_unzip 2dheat_diffusion.zip

De precisarlo, podemos visualizar los achivos del modelo tal como fue mostrado más arriba.

### Simulación con CD++

Vamos a simular 2 min de la difusión:

In [ ]:
%cdpp_run -m HeatMA.ma   \
          -l Heat.log \
          -t 00:02:00:000

### Análisis de los resultados

Veamos el contenido de los archivos `.log`.

In [ ]:
log_heat = %parse_log Heat.log

Veamos el log de la celda (5,5) que está conectada a la fuente de calor:

In [ ]:
df_cell55 = log_heat['superficie(5,5)']
df_cell55

Veamos el achivo de log de la celda (8,8) que está conectada a la fuente de frío:

In [ ]:
df_cell88 = log_heat['superficie(8,8)']
df_cell88

Veamos los eventos de calor y frío. Nos interesa mirar los valores de los puertos `in` de ambos componentes, y como nuestro graficador sabe distinguir por puerto, podemos modificar los valores de puerto en los dataframes. Luego concatenamos ambos dataframes y lo graficamos:

In [ ]:
# me quedo con los eventos de entrada
ev88 = filter_and_name(df_cell88,'X')
ev55 = filter_and_name(df_cell55,'X')

# modifico el valor del puerto 
ev88[PORT_COL] = ev88[PORT_COL].apply(lambda x: x + "Frio")
ev55[PORT_COL] = ev55[PORT_COL].apply(lambda x: x + "Calor")

# concateno dataframes
frio_y_calor = pd.concat([ev88,ev55])

# grafico los valores de los puertos in
do_chart(frio_y_calor,"stem",['inFrio','inCalor'])

Veamos el log del acoplado del modelo CellDEVS:

In [ ]:
df_cells= log_heat['superficie']
df_cells

Por último, queremos graficar los mensajes Y de las celdas (0,0), (5,5) y (8,8).

In [ ]:
# filtro por mensajes Y
superficie = filter_and_name(df_cells,'Y')

# separo por el componente que me interesa
data00 = superficie.loc[superficie[MODEL_ORIGIN_COL] == "superficie(0,0)(02)"]
data55 = superficie.loc[superficie[MODEL_ORIGIN_COL] == "superficie(5,5)(57)"]
data88 = superficie.loc[superficie[MODEL_ORIGIN_COL] == "superficie(8,8)(90)"]

# grafico
do_chart(data00)
do_chart(data55)
do_chart(data88)

### Visualización usando drawlog

Utilicemos la herramienta `drawlog` para visualizar los resultados. 

In [ ]:
%drawlog_run -mHeatMA.ma   \
             -llog.log \
             -csuperficie \
             -w6 \
             -p1 \
             -i00:00:05:000

## Otros magics

Además de los comandos y métodos que hemos presentado hasta ahora, también ofrecemos los siguientes:


* `%cdpp_recompile`: Este magic limpia la compilación y la ejecuta de nuevo (como hacer make clean y luego make desde una terminal).
* `%cdpp_download`: Este magic recibe como parámetro una url de un archivo y lo descarga con el nombre indicado. Funciona como `%cdpp_download_carleton`, con la diferencia que este último ya tiene preseteada la url.
* `%cdpp_copy_to_project`: Este magic recibe como parámetro un path a un directorio, y copia su contenido al directorio establecido previamente con `%cdpp_set_project`.  Esto es útil cuando estamos desarrollando un modelo fuera del directorio `examples` del simulador. Con esto, un posible workflow podría ser el siguiente:

# Conclusiones

En este trabajo hemos desarrollado un conjunto de métodos y funciones magic para simplificar la utilización de CD++ en JupyterLab. Como se puede observar en este notebook, estos nuevos comandos permiten una mejor legibilidad y comprensión del proceso mediante el cual se construye un modelo, se corre una simulación, y se analizan los resultados obtenidos. De esta manera, una persona que tome el curso SED podrá enfocarse en los conceptos principales, sin tener que invertir mayor tiempo en el manejo de bash desde celdas del Jupyter Notebook, o en el procesamiento de dataframes. Además, el código presente en el notebook queda mucho más limpio y permite un mejor seguimiento de la clase.

# Referencias
[1] https://ipython.readthedocs.io/en/stable/interactive/magics.html